In [1]:
# load model hyper params
# load some test image
# load some interactive window & plot it
# do it for changing one codeword at a fixed position across codebook
# etc.

In [9]:
import yaml

def update_yaml(yaml_file_path = "/home/novakovm/iris/MILOS/",
                yaml_file_name = "toy_shapes_config",
                key = "NUM_WORKERS", 
                value = 5,
                source = "training_hyperparams"):
    # make a new yaml config file from human readable version of that same file
    yaml_full_src_path = yaml_file_path + yaml_file_name+ "_human_readable" +".yaml"
    yaml_full_dsc_path = yaml_file_path + yaml_file_name +".yaml"
    data_dict = yaml.load(open(yaml_full_src_path, 'r'), Loader=yaml.FullLoader)
    with open(yaml_full_dsc_path, 'w') as yaml_file:
        yaml_file.write( yaml.dump(data_dict, default_flow_style=False))
    
    # update the newly created non-human readable file
    data_dict = yaml.load(open(yaml_full_dsc_path, 'r'), Loader=yaml.FullLoader)
    # pick the desired source list in the data dict. yaml file
    data_source_list = data_dict[source]
    # find the index in the list where the desired key is
    data_source_list_key_index = [idx for idx, data_key in enumerate(data_source_list) if key in data_key][0]
    # update step
    data_source_list[data_source_list_key_index][key] = value
    # save the change
    with open(yaml_full_dsc_path, 'w') as yaml_file:
        yaml_file.write( yaml.dump(data_dict, default_flow_style=False))
update_yaml(yaml_file_path = "/home/novakovm/iris/MILOS/",
            yaml_file_name = "toy_shapes_config",
            key = "NUM_WORKERS", 
            value = 5)
update_yaml(yaml_file_path = "/home/novakovm/iris/MILOS/",
            yaml_file_name = "crafter_config",
            key = "NUM_WORKERS", 
            value = 5)

In [2]:
from IPython.display import clear_output
from run_pipeline import run
import torch
import numpy as np
import imageio
clear_output(wait=True)
#clear_output(wait=True)


/home/novakovm/miniconda3/envs/iris_mn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-20 09:37:40.339510: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-20 09:37:40.807260: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-20 09:37:40.807307: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer

In [3]:
def change_one_token(trainers, dataset_name = "toy_dataset", dataset_type = "test"):
    M = trainers[dataset_name].model.VQ.M
    K = trainers[dataset_name].model.VQ.K

    image_batch, image_id_batch = next(iter(trainers[dataset_name].loaders[dataset_type]))
    image_batch,image_id_batch = image_batch.to(trainers[dataset_name].device), image_id_batch.to(trainers[dataset_name].device)

    trainers[dataset_name].model.VQ.output_whole_quantization_process = True
    e_and_q_latent_loss, Zq, e_latent_loss, q_latent_loss, estimate_codebook_words, encoding_indices, estimate_codebook_words_freq, estimate_codebook_words_prob, inputs, D  = \
        trainers[dataset_name].model.VQ(trainers[dataset_name].model.encoder(image_batch))
    trainers[dataset_name].model.VQ.output_whole_quantization_process = False
    
    changed_token_map_position_range_step = 1
    changed_token_map_position_range = np.arange(0,K,changed_token_map_position_range_step)
    for changed_token_map_position_row in np.arange(M+1):
        for changed_token_map_position_column in np.arange(M+1):
            
            digit_size = len(str(len(trainers[dataset_name].loaders[dataset_type].dataset)))
            
            for index_, changed_token_map_position_value in enumerate(changed_token_map_position_range):
                new_encoding_indices = encoding_indices.clone().detach().view(M+1,M+1)
                new_encoding_indices[changed_token_map_position_row, changed_token_map_position_column] = changed_token_map_position_value
                new_encoding_indices = new_encoding_indices.view(-1,1)
                trainers[dataset_name].original_reconstructed_changed_reconstucted_tokens_changed_tokens(  image_batch,# one image, it is the tensor of shape = (1,C,H,W)
                                                                                                            image_id_batch,#id
                                                                                                            new_encoding_indices, 
                                                                                                            index_,
                                                                                                            dataset_str = dataset_type,
                                                                                                            create_plot_for_every_image_in_dataset = False, 
                                                                                                            jupyter_show_images = False)
            frames_per_second = 5#10#5#10#1
            format_list = ['mp4', 'gif']
            for format_ in format_list:
                with imageio.get_writer(trainers[dataset_name].visualize_tokens_path + f"_{str(changed_token_map_position_row).zfill(1)}_x_{str(changed_token_map_position_column).zfill(1)}_token_changed_{format_}.{format_}", mode='I', fps = frames_per_second) as writer:
                    for index_, changed_token_map_position_value in enumerate(changed_token_map_position_range):
                        filename = trainers[dataset_name].all_images_full_path + f"{str(index_).zfill(digit_size)}_custom_image.png"
                        image = imageio.imread(filename)
                        writer.append_data(image)
                    writer.close()

In [4]:
trainers = {}


In [5]:
config_paths = {"toy_dataset" : "/home/novakovm/iris/MILOS/toy_shapes_config.yaml",
                "crafter_dataset" : "/home/novakovm/iris/MILOS/crafter_config.yaml"}

In [6]:
#trainers['toy_dataset'] = run(config_paths["toy_dataset"])

In [7]:
trainers['crafter_dataset'] = run(config_paths["crafter_dataset"])

Total Trainable Params in thousands: 1765

Total Memory =  25.43 GB.
Reserved Memory =  0.00 GB.
Allocated inside Reserved Memory =  0.00 GB.
Free inside Reserved Memory =  0.00 GB.

Global Free memory on the cuda:0 =  22.58 GB.
Total GPU Memory occupied on the cuda:0 =  25.43 GB.

Inverse of training data variance term is equal to =  3.9
Testing Started
Average test total loss =  6742 e-6
Average test total reconstruction loss =  4287 e-6 (64 %)
Average test total commitment loss =  491 e-6 (7 %)
Average test total VQ loss =  1964 e-6 (29 %)
Testing Ended


In [8]:
change_one_token(trainers, dataset_name = "crafter_dataset", dataset_type = "test")

/tmp/ipykernel_2192295/4248046032.py:37: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (3000, 800) to (3008, 800) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (3000, 800) to (3008, 800) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: 

: 

: 

In [ ]:
print("Finish")